<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_28_Explore_and_Search_Conversations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how Graphlit supports semantic search over conversations, not just contents. It also shows how to locate similar conversations to a given conversation.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [21]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [22]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [23]:
from typing import List, Optional

async def ingest_uri(uri: str):
    if graphlit.client is None:
        return None

    try:
        # Using synchronous mode, so the notebook waits for the content to be ingested
        response = await graphlit.client.ingest_uri(uri=uri, is_synchronous=True)

        return response.ingest_uri.id if response.ingest_uri is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def create_anthropic_specification(model: enums.AnthropicModels):
    if graphlit.client is None:
        return None

    input = input_types.SpecificationInput(
        name=f"Anthropic [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.ANTHROPIC,
        anthropic=input_types.AnthropicModelPropertiesInput(
            model=model,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_conversation(specification_id: str):
    if graphlit.client is None:
        return None

    input = input_types.ConversationInput(
        name="Conversation",
        specification=input_types.EntityReferenceInput(
            id=specification_id
        )
    )

    try:
        response = await graphlit.client.create_conversation(input)

        return response.create_conversation.id if response.create_conversation is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_conversation(conversation_id: str):
    if graphlit.client is None:
        return None

    if conversation_id is not None:
        _ = await graphlit.client.delete_conversation(conversation_id)

async def prompt_conversation(conversation_id: str, prompt: str):
    if graphlit.client is None:
        return None

    try:
        response = await graphlit.client.prompt_conversation(prompt, conversation_id)

        return response.prompt_conversation.message.message if response.prompt_conversation is not None and response.prompt_conversation.message is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def query_conversations(search: str):
    if graphlit.client is None:
        return None;

    try:
        response = await graphlit.client.query_conversations(input_types.ConversationFilter(
            search=search,
            searchType=enums.SearchTypes.HYBRID
        ))

        return response.conversations.results if response.conversations is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def query_similar_conversations(conversation_id: str):
    if graphlit.client is None:
        return None;

    try:
        response = await graphlit.client.query_conversations(input_types.ConversationFilter(
            searchType=enums.SearchTypes.VECTOR,
            conversations=[
                input_types.EntityReferenceFilter(
                    id=conversation_id
                )
            ] if conversation_id is not None else None
        ))

        return response.conversations.results if response.conversations is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_specifications():
    if graphlit.client is None:
        return None

    _ = await graphlit.client.delete_all_specifications(is_synchronous=True)

async def delete_all_conversations():
    if graphlit.client is None:
        return None

    _ = await graphlit.client.delete_all_conversations(is_synchronous=True)

async def delete_all_contents():
    if graphlit.client is None:
        return None

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)


Execute Graphlit example

In [24]:
from IPython.display import display, Markdown, HTML
import time

# Remove any existing contents, conversations and specifications; only needed for notebook example
await delete_all_conversations()
await delete_all_specifications()
await delete_all_contents()

print('Deleted all contents, conversations and specifications.')

content_id = await ingest_uri(uri="https://graphlitplatform.blob.core.windows.net/samples/Unstructured%20Data%20is%20Dark%20Data%20Podcast.mp3")

if content_id is not None:
    print(f'Ingested content [{content_id}]:')

Deleted all contents, conversations and specifications.
Ingested content [c58d39c9-3fc0-4ba0-9685-86e1820cfcfc]:


Create conversation using Anthropic Sonnet 3.5 specification.

In [25]:
    specification_id = await create_anthropic_specification(enums.AnthropicModels.CLAUDE_3_5_SONNET)

    if specification_id is not None:
        print(f'Created specification [{specification_id}].')

        conversation_id = await create_conversation(specification_id)

        if conversation_id is not None:
            print(f'Created conversation [{conversation_id}].')

            prompt = "In 3-5 detailed paragraphs, explain unstructured data and its usefulness for knowledge capture and retrieval."

            message = await prompt_conversation(conversation_id, prompt)

            if message is not None:
                display(Markdown('### Conversation:'))
                display(Markdown(f'**User:**\n{prompt}'))
                display(Markdown(f'**Assistant:**\n{message}'))
                print()

            prompt = "Can you go into more details about knowledge graphs?"

            message = await prompt_conversation(conversation_id, prompt)

            if message is not None:
                display(Markdown('### Conversation:'))
                display(Markdown(f'**User:**\n{prompt}'))
                display(Markdown(f'**Assistant:**\n{message}'))
                print()

            prompt = "OK, but be more specific."

            message = await prompt_conversation(conversation_id, prompt)

            if message is not None:
                display(Markdown('### Conversation:'))
                display(Markdown(f'**User:**\n{prompt}'))
                display(Markdown(f'**Assistant:**\n{message}'))
                print()


Created specification [7a67108a-37da-453f-92f0-ed47c4ab11c6].
Created conversation [ee21a8be-1f41-48a0-b103-a57558b98cbc].


### Conversation:

**User:**
In 3-5 detailed paragraphs, explain unstructured data and its usefulness for knowledge capture and retrieval.

**Assistant:**
Unstructured data refers to a broad set of file-based information that doesn't fit neatly into traditional structured databases. This includes images, audio, video, 3D geometry, point clouds, documents, and emails. While these files do have internal structures, they are considered 'unstructured' because their content is not easily parsed or queried like structured data. Unstructured data often contains rich, contextual information about real-world assets, people, places, and things, making it a valuable but challenging source of knowledge.

To extract value from unstructured data, it's processed through multiple layers of analysis. First-order metadata comes directly from file headers or embedded information. Second-order metadata is derived through techniques like object detection in images or term extraction from documents. Third-order metadata involves making inferences and connections, such as linking detected objects to entries in other databases. This layered approach allows for increasingly sophisticated knowledge capture, moving from basic file attributes to complex relationships and insights.

The power of unstructured data lies in its ability to be contextualized and connected through knowledge graphs. By extracting observations and creating relationships between different pieces of information, organizations can discover trends, perform semantic searches, and gain insights that were previously hidden. This approach allows for pivoting on any entity or relationship, enabling dynamic exploration of data across time and space. The result is a rich, interconnected web of knowledge that can drive more informed decision-making and uncover valuable patterns in historical and current data.

### Conversation:

**User:**
Can you go into more details about knowledge graphs?

**Assistant:**
Knowledge graphs are a powerful way to represent and connect data dynamically. Unlike traditional databases with rigid schemas, knowledge graphs allow for flexible relationships between entities through edges that can be created on-the-fly. This enables pivoting on any entity or relationship to discover new insights and connections.

The structure of a knowledge graph consists of nodes (entities) connected by edges (relationships). Entities can represent people, places, things, or concepts. The ability to invent new edges between entities allows for continuous enrichment of the graph as new connections are discovered or inferred.

Knowledge graphs excel at contextualizing and connecting unstructured data. By extracting entities and relationships from various sources like documents, images, and audio, a rich web of interconnected information can be built. This allows for semantic searches and discovery of non-obvious patterns across large datasets.

One key advantage of knowledge graphs is their ability to provide multiple layers of metadata. First-order metadata comes directly from file headers, second-order from content analysis, and third-order from inferences and connections to external data sources. This layered approach enables increasingly sophisticated knowledge capture and insights.

Knowledge graphs can continuously expand through web spidering techniques, following links to discover new related information. However, it's important to implement logic to avoid infinite loops and determine when to stop expanding based on diminishing returns of new, unique data.

### Conversation:

**User:**
OK, but be more specific.

**Assistant:**
Unstructured data encompasses a wide range of file-based information including images, audio, video, 3D geometry, point clouds, documents, and emails. While these files have internal structures, they are considered 'unstructured' because their content is not easily parsed or queried like traditional structured databases.

To extract value from unstructured data, it's processed through multiple layers of analysis. First-order metadata comes directly from file headers or embedded information. Second-order metadata is derived through techniques like object detection in images or term extraction from documents. Third-order metadata involves making inferences and connections to external data sources.

The power of unstructured data lies in its ability to be contextualized and connected through knowledge graphs. By extracting observations and creating relationships between different pieces of information, organizations can discover trends, perform semantic searches, and gain insights that were previously hidden in their data.

Knowledge graphs excel at representing and connecting data dynamically. Unlike traditional databases with rigid schemas, knowledge graphs allow for flexible relationships between entities through edges that can be created on-the-fly. This enables pivoting on any entity or relationship to discover new insights and connections.

One key advantage of knowledge graphs is their ability to provide multiple layers of metadata and contextualization. This layered approach enables increasingly sophisticated knowledge capture, moving from basic file attributes to complex relationships and insights across large datasets.

Create another conversation, using same specification.

In [26]:
        conversation_id = await create_conversation(specification_id)

        if conversation_id is not None:
            print(f'Created conversation [{conversation_id}].')

            prompt = "Extract the names of any people mentioned in this podcast."

            message = await prompt_conversation(conversation_id, prompt)

            if message is not None:
                display(Markdown('### Conversation:'))
                display(Markdown(f'**User:**\n{prompt}'))
                display(Markdown(f'**Assistant:**\n{message}'))
                print()

            prompt = "Can you go into more details about their background or where they work?"

            message = await prompt_conversation(conversation_id, prompt)

            if message is not None:
                display(Markdown('### Conversation:'))
                display(Markdown(f'**User:**\n{prompt}'))
                display(Markdown(f'**Assistant:**\n{message}'))
                print()

            prompt = "OK, describe the topics each of these people may be most interested in."

            message = await prompt_conversation(conversation_id, prompt)

            if message is not None:
                display(Markdown('### Conversation:'))
                display(Markdown(f'**User:**\n{prompt}'))
                display(Markdown(f'**Assistant:**\n{message}'))
                print()


Created conversation [b20f72d8-4a9a-49fa-b8e2-299e510e63b9].


### Conversation:

**User:**
Extract the names of any people mentioned in this podcast.

**Assistant:**
Kirk Marple, founder and CEO of Unstruct Data

Daniel, host of the Mapscaping podcast

Lizzie, a new supporter of the podcast on Patreon

### Conversation:

**User:**
Can you go into more details about their background or where they work?

**Assistant:**
Kirk Marple is the founder and CEO of Unstruct Data, a company that launched about a month prior to the podcast recording. He has a background as a long-time software developer, with experience in the media software space and dabbling in geospatial data throughout his career.

Daniel is the host of the Mapscaping podcast, which focuses on the geospatial community. He conducts interviews with guests and manages the podcast's content and distribution.

Lizzie is mentioned as a new supporter of the Mapscaping podcast on Patreon, but no further details about her background or work are provided in the given context.

### Conversation:

**User:**
OK, describe the topics each of these people may be most interested in.

**Assistant:**
Kirk Marple: Unstructured data analysis, knowledge graphs, geospatial data integration, data enrichment techniques, and enterprise software solutions for managing and extracting insights from diverse data types like images, audio, and documents.

Daniel: Geospatial technologies, mapping applications, podcast production and distribution, and interviewing experts in the geospatial field to share insights with the Mapscaping podcast audience.

Lizzie: As a Patreon supporter, Lizzie likely has an interest in geospatial topics, mapping technologies, and staying informed about developments in the geospatial community through podcast content.

Create one more conversation, unrelated to the others.

In [27]:
        conversation_id = await create_conversation(specification_id)

        if conversation_id is not None:
            print(f'Created conversation [{conversation_id}].')

            prompt = "Tell me fun facts about Seattle"

            message = await prompt_conversation(conversation_id, prompt)

            if message is not None:
                display(Markdown('### Conversation:'))
                display(Markdown(f'**User:**\n{prompt}'))
                display(Markdown(f'**Assistant:**\n{message}'))
                print()

            prompt = "Name some of the major waterways"

            message = await prompt_conversation(conversation_id, prompt)

            if message is not None:
                display(Markdown('### Conversation:'))
                display(Markdown(f'**User:**\n{prompt}'))
                display(Markdown(f'**Assistant:**\n{message}'))
                print()

            prompt = "What are its major sports teams?"

            message = await prompt_conversation(conversation_id, prompt)

            if message is not None:
                display(Markdown('### Conversation:'))
                display(Markdown(f'**User:**\n{prompt}'))
                display(Markdown(f'**Assistant:**\n{message}'))
                print()


Created conversation [d88456fa-acae-4a47-b925-bb02ef586a2c].


### Conversation:

**User:**
Tell me fun facts about Seattle

**Assistant:**
Seattle is home to the first Starbucks coffee shop, opened in 1971 at Pike Place Market. The city is known for its thriving coffee culture and has more coffee shops per capita than any other U.S. city.

The Space Needle, Seattle's iconic landmark, was built for the 1962 World's Fair. It stands 605 feet tall and offers panoramic views of the city, Puget Sound, and surrounding mountains.

Seattle's famous Pike Place Market is one of the oldest continuously operated public farmers' markets in the United States, having opened in 1907. It's known for its flying fish tradition, where fishmongers toss fish to each other as part of the show.

The city experiences an average of 152 rainy days per year, contributing to its nickname 'The Emerald City' due to the lush green vegetation that thrives in the mild, moist climate.

Seattle is the birthplace of grunge music, with iconic bands like Nirvana, Pearl Jam, and Soundgarden originating from the city's music scene in the 1990s.

### Conversation:

**User:**
Name some of the major waterways

**Assistant:**
Seattle is surrounded by several major waterways, including Puget Sound, which is an inlet of the Pacific Ocean. This expansive body of water is crucial for the city's maritime activities and scenic beauty.

Lake Washington, a large freshwater lake, borders Seattle to the east. It's connected to Puget Sound via the Lake Washington Ship Canal, which includes the Hiram M. Chittenden Locks, also known as the Ballard Locks.

The Duwamish River flows through Seattle and empties into Elliott Bay, part of Puget Sound. This industrialized waterway has played a significant role in the city's economic development.

Lake Union, located within the city limits, is another notable waterway. It's popular for recreational activities and is home to many houseboats, giving Seattle a unique floating neighborhood.

The Green River, which becomes the Duwamish River as it enters Seattle, is an important waterway for the region's ecology and has historical significance for local Native American tribes.

### Conversation:

**User:**
What are its major sports teams?

**Assistant:**
Seattle Seahawks: NFL team known for their passionate '12th Man' fanbase and Super Bowl XLVIII victory in 2014.

Seattle Mariners: MLB team featuring iconic players like Ken Griffey Jr. and Ichiro Suzuki in their history.

Seattle Kraken: NHL expansion team that began play in 2021, quickly gaining popularity in the hockey-loving city.

Seattle Sounders FC: MLS team with a strong following, multiple MLS Cup wins, and consistently high attendance.

Seattle Storm: WNBA team boasting multiple championships and star players like Sue Bird and Breanna Stewart.

OL Reign: NWSL team (formerly Seattle Reign FC) featuring top international talent in women's soccer.

In [28]:
search = 'Mapscaping podcast'

conversations = await query_conversations(search)

if conversations is not None and len(conversations) > 0:
    for conversation in conversations:
        if conversation is not None:
            display(Markdown(f'### Found conversation [{conversation.id}], relevance [{conversation.relevance}]:'))

            if conversation.messages is not None:
                for message in conversation.messages:
                    if message is not None:
                        display(Markdown(f'**{message.role}:**\n{message.message}'))

            print()
else:
    print(f'No conversations found for search [{search}].')

### Found conversation [b20f72d8-4a9a-49fa-b8e2-299e510e63b9], relevance [0.03333333507180214]:

**USER:**
Extract the names of any people mentioned in this podcast.

**ASSISTANT:**
Kirk Marple, founder and CEO of Unstruct Data

Daniel, host of the Mapscaping podcast

Lizzie, a new supporter of the podcast on Patreon

**USER:**
Can you go into more details about their background or where they work?

**ASSISTANT:**
Kirk Marple is the founder and CEO of Unstruct Data, a company that launched about a month prior to the podcast recording. He has a background as a long-time software developer, with experience in the media software space and dabbling in geospatial data throughout his career.

Daniel is the host of the Mapscaping podcast, which focuses on the geospatial community. He conducts interviews with guests and manages the podcast's content and distribution.

Lizzie is mentioned as a new supporter of the Mapscaping podcast on Patreon, but no further details about her background or work are provided in the given context.

**USER:**
OK, describe the topics each of these people may be most interested in.

**ASSISTANT:**
Kirk Marple: Unstructured data analysis, knowledge graphs, geospatial data integration, data enrichment techniques, and enterprise software solutions for managing and extracting insights from diverse data types like images, audio, and documents.

Daniel: Geospatial technologies, mapping applications, podcast production and distribution, and interviewing experts in the geospatial field to share insights with the Mapscaping podcast audience.

Lizzie: As a Patreon supporter, Lizzie likely has an interest in geospatial topics, mapping technologies, and staying informed about developments in the geospatial community through podcast content.

In [31]:
search = 'Knowledge graphs'

conversations = await query_conversations(search)

if conversations is not None and len(conversations) > 0:
    for conversation in conversations:
        if conversation is not None:
            display(Markdown(f'### Found conversation [{conversation.id}], relevance [{conversation.relevance}]:'))

            if conversation.messages is not None:
                for message in conversation.messages:
                    if message is not None:
                        display(Markdown(f'**{message.role}:**\n{message.message}'))

            print()
else:
    print(f'No conversations found for search [{search}].')

### Found conversation [ee21a8be-1f41-48a0-b103-a57558b98cbc], relevance [0.03333333507180214]:

**USER:**
In 3-5 detailed paragraphs, explain unstructured data and its usefulness for knowledge capture and retrieval.

**ASSISTANT:**
Unstructured data refers to a broad set of file-based information that doesn't fit neatly into traditional structured databases. This includes images, audio, video, 3D geometry, point clouds, documents, and emails. While these files do have internal structures, they are considered 'unstructured' because their content is not easily parsed or queried like structured data. Unstructured data often contains rich, contextual information about real-world assets, people, places, and things, making it a valuable but challenging source of knowledge.

To extract value from unstructured data, it's processed through multiple layers of analysis. First-order metadata comes directly from file headers or embedded information. Second-order metadata is derived through techniques like object detection in images or term extraction from documents. Third-order metadata involves making inferences and connections, such as linking detected objects to entries in other databases. This layered approach allows for increasingly sophisticated knowledge capture, moving from basic file attributes to complex relationships and insights.

The power of unstructured data lies in its ability to be contextualized and connected through knowledge graphs. By extracting observations and creating relationships between different pieces of information, organizations can discover trends, perform semantic searches, and gain insights that were previously hidden. This approach allows for pivoting on any entity or relationship, enabling dynamic exploration of data across time and space. The result is a rich, interconnected web of knowledge that can drive more informed decision-making and uncover valuable patterns in historical and current data.

**USER:**
Can you go into more details about knowledge graphs?

**ASSISTANT:**
Knowledge graphs are a powerful way to represent and connect data dynamically. Unlike traditional databases with rigid schemas, knowledge graphs allow for flexible relationships between entities through edges that can be created on-the-fly. This enables pivoting on any entity or relationship to discover new insights and connections.

The structure of a knowledge graph consists of nodes (entities) connected by edges (relationships). Entities can represent people, places, things, or concepts. The ability to invent new edges between entities allows for continuous enrichment of the graph as new connections are discovered or inferred.

Knowledge graphs excel at contextualizing and connecting unstructured data. By extracting entities and relationships from various sources like documents, images, and audio, a rich web of interconnected information can be built. This allows for semantic searches and discovery of non-obvious patterns across large datasets.

One key advantage of knowledge graphs is their ability to provide multiple layers of metadata. First-order metadata comes directly from file headers, second-order from content analysis, and third-order from inferences and connections to external data sources. This layered approach enables increasingly sophisticated knowledge capture and insights.

Knowledge graphs can continuously expand through web spidering techniques, following links to discover new related information. However, it's important to implement logic to avoid infinite loops and determine when to stop expanding based on diminishing returns of new, unique data.

...

**ASSISTANT:**
Kirk Marple, founder and CEO of Unstruct Data

Daniel, host of the Mapscaping podcast

Lizzie, a new supporter of the podcast on Patreon

**USER:**
Can you go into more details about their background or where they work?

**ASSISTANT:**
Kirk Marple is the founder and CEO of Unstruct Data, a company that launched about a month prior to the podcast recording. He has a background as a long-time software developer, with experience in the media software space and dabbling in geospatial data throughout his career.

Daniel is the host of the Mapscaping podcast, which focuses on the geospatial community. He conducts interviews with guests and manages the podcast's content and distribution.

Lizzie is mentioned as a new supporter of the Mapscaping podcast on Patreon, but no further details about her background or work are provided in the given context.

**USER:**
OK, describe the topics each of these people may be most interested in.

**ASSISTANT:**
Kirk Marple: Unstructured data analysis, knowledge graphs, geospatial data integration, data enrichment techniques, and enterprise software solutions for managing and extracting insights from diverse data types like images, audio, and documents.

Daniel: Geospatial technologies, mapping applications, podcast production and distribution, and interviewing experts in the geospatial field to share insights with the Mapscaping podcast audience.

Lizzie: As a Patreon supporter, Lizzie likely has an interest in geospatial topics, mapping technologies, and staying informed about developments in the geospatial community through podcast content.

Search for similar conversation. Note that the conversations return their relevance on 0-1 scale, based on vector embedding similarity.  You can develop a cutoff threshold for similarity, or just take the most similar conversation, depending on your use case.

In [30]:
conversation_id = conversations[0].id if conversations is not None and len(conversations) > 0 else None

if conversation_id is not None:
    similar_conversations = await query_similar_conversations(conversation_id)

    if similar_conversations is not None and len(similar_conversations) > 0:
        for conversation in similar_conversations:
            if conversation is not None and conversation.id != conversation_id: # ignore same conversation
                display(Markdown(f'### Found similar conversation [{conversation.id}] to [{conversation_id}], relevance [{conversation.relevance}]:'))

                if conversation.messages is not None:
                    for message in conversation.messages:
                        if message is not None:
                            display(Markdown(f'**{message.role}:**\n{message.message}'))


### Found similar conversation [b20f72d8-4a9a-49fa-b8e2-299e510e63b9] to [ee21a8be-1f41-48a0-b103-a57558b98cbc], relevance [0.8349484]:

**USER:**
Extract the names of any people mentioned in this podcast.

**ASSISTANT:**
Kirk Marple, founder and CEO of Unstruct Data

Daniel, host of the Mapscaping podcast

Lizzie, a new supporter of the podcast on Patreon

**USER:**
Can you go into more details about their background or where they work?

**ASSISTANT:**
Kirk Marple is the founder and CEO of Unstruct Data, a company that launched about a month prior to the podcast recording. He has a background as a long-time software developer, with experience in the media software space and dabbling in geospatial data throughout his career.

Daniel is the host of the Mapscaping podcast, which focuses on the geospatial community. He conducts interviews with guests and manages the podcast's content and distribution.

Lizzie is mentioned as a new supporter of the Mapscaping podcast on Patreon, but no further details about her background or work are provided in the given context.

**USER:**
OK, describe the topics each of these people may be most interested in.

**ASSISTANT:**
Kirk Marple: Unstructured data analysis, knowledge graphs, geospatial data integration, data enrichment techniques, and enterprise software solutions for managing and extracting insights from diverse data types like images, audio, and documents.

Daniel: Geospatial technologies, mapping applications, podcast production and distribution, and interviewing experts in the geospatial field to share insights with the Mapscaping podcast audience.

Lizzie: As a Patreon supporter, Lizzie likely has an interest in geospatial topics, mapping technologies, and staying informed about developments in the geospatial community through podcast content.